In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [2]:
accounts = pd.read_csv("accounts.csv")
journal = pd.read_csv("journal.csv")

In [3]:
accounts.drop(columns="owner_id" , inplace=True)

In [4]:
journal.drop(columns="owner_id" , inplace=True)

In [5]:
accounts

,id,account,normal_balance,account_type
0,8,Cash,Debit,Assest
1,9,Capital,Credit,Investment
2,10,Computer equipment,Debit,Assest
3,11,Supplier,Debit,Assest
4,12,Account Payable,Credit,liabilities
5,13,Service Revenue,Credit,Revenue
6,14,Advertise expenses,Debit,Expenses
7,15,Advertise Payable,Credit,liabilities
8,16,Account Receivable,Debit,Assest
9,17,Office Rent,Debit,Expenses


In [6]:
journal

,id,account_id,date,balance,transaction_type,comment
0,8,8,2020-06-17,15000.0,Debit,NaN
1,9,9,2020-06-18,15000.0,Credit,NaN
2,10,10,2020-06-17,7000.0,Debit,NaN
3,11,8,2020-06-17,7000.0,Credit,NaN
4,12,11,2020-06-17,1600.0,Debit,NaN
5,13,12,2020-06-17,1600.0,Credit,NaN
6,14,8,2020-06-17,1200.0,Debit,NaN
7,15,13,2020-06-17,1200.0,Credit,NaN
8,16,14,2020-06-17,250.0,Debit,NaN
9,17,15,2020-06-17,250.0,Credit,NaN


# merge data frame

In [7]:
data = accounts.merge(journal , left_on="id" , right_on = "account_id" , how="outer")

In [8]:
data.head()

,id_x,account,normal_balance,account_type,id_y,account_id,date,balance,transaction_type,comment
0,8,Cash,Debit,Assest,8,8,2020-06-17,15000.0,Debit,NaN
1,8,Cash,Debit,Assest,11,8,2020-06-17,7000.0,Credit,NaN
2,8,Cash,Debit,Assest,14,8,2020-06-17,1200.0,Debit,NaN
3,8,Cash,Debit,Assest,18,8,2020-06-17,1200.0,Debit,NaN
4,8,Cash,Debit,Assest,24,8,2020-06-17,1700.0,Debit,NaN


In [9]:
data.shape

(24, 10)

In [10]:
# return True or false if transaction_type == Normal Balance
data["helper1"] = data["normal_balance"] == data["transaction_type"]
#convert True, False to 1,-1 respectively
data["helper1"] = data["helper1"].replace([True, False] , [1,-1])
#convert balance into negative in case the transaction
data["balance_negative"] = data["helper1"] * data["balance"]


data.head()

,id_x,account,normal_balance,account_type,id_y,account_id,date,balance,transaction_type,comment,helper1,balance_negative
0,8,Cash,Debit,Assest,8,8,2020-06-17,15000.0,Debit,NaN,1,15000.0
1,8,Cash,Debit,Assest,11,8,2020-06-17,7000.0,Credit,NaN,-1,-7000.0
2,8,Cash,Debit,Assest,14,8,2020-06-17,1200.0,Debit,NaN,1,1200.0
3,8,Cash,Debit,Assest,18,8,2020-06-17,1200.0,Debit,NaN,1,1200.0
4,8,Cash,Debit,Assest,24,8,2020-06-17,1700.0,Debit,NaN,1,1700.0


In [11]:
def prepare_trial_balance(df):
    trial_balance = df.pivot_table(values="balance_negative" , index="account" , columns="normal_balance" ,aggfunc=np.sum, fill_value=0)
    return trial_balance , trial_balance.sum()
prepare_trial_balance(data)

(normal_balance        Credit  Debit
 account                            
 Account Payable         1600      0
 Account Receivable         0   1400
 Advertise Payable          0      0
 Advertise expenses         0    250
 Capital                15000      0
 Cash                       0  12050
 Computer equipment         0   7000
 Office Rent                0    600
 Salaries of employee       0    900
 Service Revenue         4700      0
 Supplier                   0   1600
 Utilities                  0    200
 Withdrawal                 0   1300,
 normal_balance
 Credit    21300
 Debit     25300
 dtype: int64)

In [12]:
def prepare_net_income(df):
    net_income = df.query('account_type == "Revenue" or account_type == "Expenses" ').pivot_table(index = "account" , columns="account_type" , values="balance_negative" , aggfunc=np.sum).sort_values('Revenue' , ascending=False)
    return net_income , net_income.sum()

prepare_net_income(data)[0]

account_type,Expenses,Revenue
account,,
Service Revenue,NaN,4700.0
Advertise expenses,250.0,NaN
Office Rent,600.0,NaN
Salaries of employee,900.0,NaN
Utilities,200.0,NaN


In [13]:
def prepare_equity_statement(df):
    investment = df.query('account_type == "Investment"')["balance_negative"].sum()
    drawings = df.query('account_type == "Drawings"')["balance_negative"].sum()
    return  investment , drawings 

prepare_equity_statement(data)

(15000.0, 1300.0)

In [14]:
def prepare_finacial_statement(df):
    assest = df.query('account_type == "Assest"').pivot_table(values="balance_negative" , index="account" , columns="normal_balance" ,aggfunc=np.sum, fill_value=0)
    total_assest= assest.sum()
    assest[""] = ""
    
    liabilities = df.query('account_type == "liabilities"').pivot_table(values="balance_negative" , index="account" , columns="normal_balance" ,aggfunc=np.sum, fill_value=0)
    total_liabilities = liabilities.sum()
    return assest , total_assest , liabilities ,total_liabilities
prepare_finacial_statement(data)

(normal_balance      Debit  
 account                    
 Account Receivable   1400  
 Cash                12050  
 Computer equipment   7000  
 Supplier             1600  ,
 normal_balance
 Debit    22050
 dtype: int64,
 normal_balance     Credit
 account                  
 Account Payable      1600
 Advertise Payable       0,
 normal_balance
 Credit    1600
 dtype: int64)

In [16]:
data.head()

,id_x,account,normal_balance,account_type,id_y,account_id,date,balance,transaction_type,comment,helper1,balance_negative
0,8,Cash,Debit,Assest,8,8,2020-06-17,15000.0,Debit,NaN,1,15000.0
1,8,Cash,Debit,Assest,11,8,2020-06-17,7000.0,Credit,NaN,-1,-7000.0
2,8,Cash,Debit,Assest,14,8,2020-06-17,1200.0,Debit,NaN,1,1200.0
3,8,Cash,Debit,Assest,18,8,2020-06-17,1200.0,Debit,NaN,1,1200.0
4,8,Cash,Debit,Assest,24,8,2020-06-17,1700.0,Debit,NaN,1,1700.0


In [19]:
# fig = go.Figure([go.Bar(x=data["account"], y=data["balance_negative"])])
# fig.show()

In [39]:
q = data.groupby("account_type")["balance_negative"].sum()
q.sort_values(ascending=False , inplace=True)


In [40]:
fig = go.Figure([go.Bar(x=q.index , y=q.values)])
fig.show()

In [32]:
q.sort_values(ascending=False)


account_type
Assest         22050.0
Investment     15000.0
Revenue         4700.0
Expenses        1950.0
liabilities     1600.0
Drawings        1300.0
Name: balance_negative, dtype: float64

In [33]:
q

account_type
Assest         22050.0
Drawings        1300.0
Expenses        1950.0
Investment     15000.0
Revenue         4700.0
liabilities     1600.0
Name: balance_negative, dtype: float64

In [45]:
data.account.unique()

array(['Cash', 'Capital', 'Computer equipment', 'Supplier',
       'Account Payable', 'Service Revenue', 'Advertise expenses',
       'Advertise Payable', 'Account Receivable', 'Office Rent',
       'Salaries of employee', 'Utilities', 'Withdrawal'], dtype=object)

In [51]:
q2 = data.query("account == 'Cash' ")
q2.head()

,id_x,account,normal_balance,account_type,id_y,account_id,date,balance,transaction_type,comment,helper1,balance_negative
0,8,Cash,Debit,Assest,8,8,2020-06-17,15000.0,Debit,NaN,1,15000.0
1,8,Cash,Debit,Assest,11,8,2020-06-17,7000.0,Credit,NaN,-1,-7000.0
2,8,Cash,Debit,Assest,14,8,2020-06-17,1200.0,Debit,NaN,1,1200.0
3,8,Cash,Debit,Assest,18,8,2020-06-17,1200.0,Debit,NaN,1,1200.0
4,8,Cash,Debit,Assest,24,8,2020-06-17,1700.0,Debit,NaN,1,1700.0


In [50]:
import plotly.express as px


In [52]:
fig = px.line(q2, x="date", y="balance_negative")
fig.show()